In [1]:
# constansts

defaultWordLength = 6

MIN_WORD_LENGTH = 4
MAX_WORD_LENGTH = 8


In [2]:

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'

def normalizeWord(word):
    return word.upper()
    
lines = []
with open(dictionaryFileName) as f:
    lines = f.read().splitlines()

filteredWords = []
for line in lines:
    length = len(line)
    if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH:
        filteredWords.append(normalizeWord(line))
    
print('{0}/{1}'.format(len(filteredWords), len(lines)))

print(filteredWords[:10])

202685/4248757
['ABABUA', 'ABACA', 'ABACETI', 'ABACETO', 'ABACHI', 'ABACISTA', 'ABACISTI', 'ABACO', 'ABADESSA', 'ABADESSE']


In [3]:


def wordsOfLength(length=5):
    return (word for word in filteredWords if len(word) == length)

len(list(wordsOfLength(defaultWordLength)))


23552

In [4]:

def isAnagram(sourceWord, targetWord):
    if len(sourceWord) != len(targetWord):
        return False
    
    res = True
    
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    #print(targetCharSet)
    for i, ch in enumerate(sourceWord):
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
        else:
            res = False
            break
    
    return res
    
        
assert(isAnagram('', ''))
assert(not isAnagram('corto', 'trOca'))
assert(not isAnagram('c', ''))
assert(isAnagram('cortao', 'trocao'))

srcWord = 'ostra'
dstWord ='motri'

print('isAnagram("{0}", "{1}")'.format(srcWord, dstWord))
print(isAnagram(srcWord, dstWord))

isAnagram("ostra", "motri")
False


In [137]:
def isAnagramEx(sourceWord, targetWord, maxDiffLength=0, strict=False):
    if maxDiffLength == 0:
        return isAnagram(sourceWord, targetWord)
    else:
        wordDiffLength = abs(len(sourceWord) - len(targetWord))
        if wordDiffLength > maxDiffLength:
            return False
        
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    if len(sourceCharSet) > len(targetCharSet):
        sourceCharSet, targetCharSet = targetCharSet, sourceCharSet
        
    #print(targetCharSet, sourceCharSet)
    for ch in sourceCharSet:
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
    
    #print(targetCharSet)
    targetCharSetLength = len(targetCharSet)
    #print(wordDiffLength, targetCharSet)
    res = (not strict and targetCharSetLength <= maxDiffLength) or (strict and targetCharSetLength == maxDiffLength)
    return res
    
        
maxDiffLength=2

assert(isAnagramEx('', ''))
assert(not isAnagramEx('corto', 'trOca'))
assert(not isAnagramEx('c', ''))
assert(isAnagramEx('cortao', 'trocao'))
assert(isAnagramEx('cortao', 'trocaox', 1))
assert(not isAnagramEx('strappo', 'posta', 1))
assert(isAnagramEx('strappo', 'posta', 2))
assert(isAnagramEx('strappo', 'posta', 3))
assert(not isAnagramEx('strappo', 'posta', 3, strict=True))
assert(isAnagramEx('strappo', 'posta', 2, strict=True))
assert(isAnagramEx('LEVI', 'ABILE', 2,))
assert(not isAnagramEx('TUFFAI', 'ABACA', 2,))


srcWord = 'straccio'
dstWord ='crosta'

print('isAnagramEx("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx(srcWord, dstWord, maxDiffLength=maxDiffLength))


isAnagramEx("straccio", "crosta", 2)
True


In [9]:
# http://stevehanov.ca/blog/index.php?id=114
# By Steve Hanov, 2011. Released to the public domain
import time
import sys

DICTIONARY = "/usr/share/dict/words";
TARGET = "SCORTA"
MAX_COST = 4

# Keep some interesting statistics
NodeCount = 0
WordCount = 0

# The Trie data structure keeps a set of words, organized with one node for
# each letter. Each node has a branch for each letter that may follow it in the
# set of words.
class TrieNode:
    def __init__(self):
        self.word = None
        self.children = {}

        global NodeCount
        NodeCount += 1

    def insert( self, word ):
        node = self
        for letter in word:
            if letter not in node.children: 
                node.children[letter] = TrieNode()

            node = node.children[letter]

        node.word = word

# read dictionary file into a trie
start = time.time()

trie = TrieNode()
for word in filteredWords:
    WordCount += 1
    trie.insert( word )

end = time.time()
print("Build took %g s" % (end - start))

print("Read %d words into %d nodes" % (WordCount, NodeCount))

# The search function returns a list of all words that are less than the given
# maximum distance from the target word
def search( word, maxCost ):

    # build first row
    currentRow = range( len(word) + 1 )

    results = []

    # recursively search each branch of the trie
    for letter in trie.children:
        searchRecursive( trie.children[letter], letter, word, currentRow, 
            results, maxCost )

    return results

# This recursive helper is used by the search function above. It assumes that
# the previousRow has been filled in already.
def searchRecursive( node, letter, word, previousRow, results, maxCost ):

    columns = len( word ) + 1
    currentRow = [ previousRow[0] + 1 ]

    # Build one row for the letter, with a column for each letter in the target
    # word, plus one for the empty string at column 0
    for column in range( 1, columns ):

        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1

        if word[column - 1] != letter:
            replaceCost = previousRow[ column - 1 ] + 1
        else:                
            replaceCost = previousRow[ column - 1 ]

        currentRow.append( min( insertCost, deleteCost, replaceCost ) )

    # if the last entry in the row indicates the optimal cost is less than the
    # maximum cost, and there is a word in this trie node, then add it.
    if currentRow[-1] <= maxCost and node.word != None:
        results.append( (node.word, currentRow[-1] ) )

    # if any entries in the row are less than the maximum cost, then 
    # recursively search each branch of the trie
    if min( currentRow ) <= maxCost:
        for letter in node.children:
            searchRecursive( node.children[letter], letter, word, currentRow, 
                results, maxCost )

start = time.time()
results = search( TARGET, MAX_COST )
end = time.time()

for result in results: print(result)

print("Search took %g s" % (end - start))



Build took 1.05505 s
Read 202685 words into 327088 nodes
('ABATA', 4)
('ABBORDA', 4)
('ABBORRA', 4)
('ABBOTTA', 4)
('ABDOTTA', 4)
('ABENTA', 4)
('ABERRA', 4)
('ABITA', 4)
('ABOLITA', 4)
('ABOLLA', 4)
('ABORRA', 3)
('ABORRE', 4)
('ABORRI', 4)
('ABORRITA', 4)
('ABORRO', 4)
('ABORTI', 3)
('ABORTII', 4)
('ABORTIR', 4)
('ABORTIRA', 4)
('ABORTITA', 4)
('ABORTIVA', 4)
('ABORTO', 3)
('ACACIA', 4)
('ACAICA', 4)
('ACANTI', 4)
('ACANTO', 4)
('ACARDIA', 4)
('ACARI', 4)
('ACARO', 4)
('ACARPA', 3)
('ACARPE', 4)
('ACARPI', 4)
('ACARPO', 4)
('ACATI', 4)
('ACATO', 4)
('ACCA', 4)
('ACCADA', 4)
('ACCANA', 4)
('ACCARNA', 4)
('ACCASA', 4)
('ACCATTA', 4)
('ACCECA', 4)
('ACCEDA', 4)
('ACCENTA', 4)
('ACCERTA', 3)
('ACCERTAI', 4)
('ACCERTAR', 4)
('ACCERTI', 4)
('ACCERTO', 4)
('ACCESA', 4)
('ACCETTA', 4)
('ACCHETA', 4)
('ACCHITA', 4)
('ACCIA', 4)
('ACCINTA', 4)
('ACCISA', 4)
('ACCOCCA', 4)
('ACCODA', 4)
('ACCODATA', 4)
('ACCOLGA', 4)
('ACCOLITA', 4)
('ACCOLLA', 4)
('ACCOLTA', 3)
('ACCOLTE', 4)
('ACCOLTI', 4)
('

In [21]:
import random

startWord = random.choice(filteredWords)
print("START WORD:", startWord)

start = time.time()
results = search(startWord, 3)
end = time.time()

print("Search took %g s" % (end - start))

validEndWord = random.choice(results)

print("END WORD:", validEndWord)


START WORD: LIANA
Search took 0.13877 s
END WORD: ('LAIDAVA', 3)


In [237]:
sixLetterWords = [word for word in filteredWords if len(word) == 6]

restart = True

while restart:
    restart = False
    startWord = random.choice(sixLetterWords)
    #startWord = 'CONDOTTE'
    print("START WORD:", startWord)

    steps = set(startWord)

    prevStep = (startWord, 0)
    for i in range(6):
        results = search(prevStep[0], 1)
        anagrams = findAnagramsEx(filteredWords, prevStep[0], maxDiffLength=1, strict=False)
        choice = random.choice(["levenshtein", "anagram"])
        if choice == "anagram":
            if len(anagrams) < 10:
                restart = True
                print(choice, len(anagrams))
                break
        else:
            if len(results) < 10:
                restart = True
                print(choice, len(results))
                break
                
        done = False
        while not done:
            if choice == "anagram":
                nextStep = random.choice(anagrams)
            else:
                nextStep = random.choice(results)

            if nextStep[0] not in steps:
                steps.add(nextStep[0])
                done = True

        resultsCount = len(results) if choice == "levenshtein" else len(anagrams)
        print(nextStep, resultsCount, choice)
        prevStep = nextStep

    print("restart", restart)



START WORD: MARCAI
('MARCAMI', 1) 22 levenshtein
('MIMARLA', 0) 181 anagram
('ALZARMI', 0) 191 anagram
('LAMIERA', 0) 144 anagram
('MERLACI', 0) 540 anagram
('MELACI', 1) 13 levenshtein
restart False


In [169]:
import random

startWord = random.choice(filteredWords)
print("START WORD:", startWord)

def findAnagramsEx(wordList, word, maxDiffLength=1, strict=False):
    results = []
    for entry in wordList:
        if word == entry:
            continue
        if isAnagramEx(entry, word, maxDiffLength=maxDiffLength, strict=strict):
            diffLength = abs(len(entry) - len(word))
            results.append((entry, diffLength))
    
    return results

anagramsDict = {0:[], 1:[], 2:[]}
randomFilteredWords = filteredWords[:]
random.shuffle(filteredWords[:])
randomFilteredWords = sorted(randomFilteredWords)

anagrams = findAnagramsEx(randomFilteredWords, startWord, maxDiffLength=1, strict=False)
for anagram in anagrams:
    anagramsDict[anagram[1]].append(anagram[0])
    
print(anagrams)
#print(anagramsDict[0][:10])

START WORD: VOGLIOSI
[('COVIGLIO', 0), ('DISGIOVO', 0), ('DISVOGLI', 0), ('DIVISOLO', 0), ('DOGLIOSI', 0), ('FOGLIOSI', 0), ('GLORIOSI', 0), ('INGOLOSI', 0), ('INVOGLIO', 0), ('IOGLOSSI', 0), ('ISTOLOGI', 0), ('LIVIDOSO', 0), ('LIVOROSI', 0), ('LOGLIOSI', 0), ('MISOLOGI', 0), ('OLIGISTO', 0), ('OLIVIGNO', 0), ('RISVOLGI', 0), ('RISVOLGO', 0), ('RIVOGLIO', 0), ('ROVIGLIO', 0), ('SILOLOGI', 0), ('SINOLOGI', 0), ('SITOLOGI', 0), ('SMOVIGLI', 0), ('SOLITOVI', 0), ('SOLIVAGO', 0), ('SOLVIGLI', 0), ('SOMIGLIO', 0), ('SVALIGIO', 0), ('SVIOLINO', 0), ('SVIRGOLI', 0), ('SVIRGOLO', 0), ('SVOGLIAI', 0), ('SVOGLICI', 0), ('SVOGLILI', 0), ('SVOGLILO', 0), ('SVOGLIMI', 0), ('SVOGLINO', 0), ('SVOGLIO', 1), ('SVOGLITI', 0), ('SVOLGICI', 0), ('SVOLGILI', 0), ('SVOLGILO', 0), ('SVOLGIMI', 0), ('SVOLGITI', 0), ('TIGLIOSO', 0), ('VAIOLOSI', 0), ('VIGOLESI', 0), ('VIGOROSI', 0), ('VINILOGO', 0), ('VIROLOGI', 0), ('VISCIOLO', 0), ('VISIGOTO', 0), ('VISTOGLI', 0), ('VOGLIOSA', 0), ('VOGLIOSE', 0), ('VOGLIOSO

In [ ]:
print(anagramsDict[1])